In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import numpy as np
import math
import panel as pn

In [ ]:
pn.extension('echarts')

In [ ]:
#model_name = "politic_EuroBERT-610m_multilabel_bundestag_and_wahlomat"
#model_name = "politic_EuroBERT-210m_multilabel_bundestag_and_wahlomat"
model_name = "../05_train_new_model/politic_gemma-2-2b_multilabel_bundestag_and_wahlomat/checkpoint-7960_export"

In [ ]:
max_length = 8192
model = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)
#tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, max_length=max_length, TOKENIZERS_PARALLELISM=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b", do_lower_case=False, max_length=max_length, TOKENIZERS_PARALLELISM=True, trust_remote_code=True)

In [ ]:
#model.push_to_hub("SinclairSchneider/german_politic_direction_EuroBERT-210m")

In [ ]:
#tokenizer.push_to_hub("SinclairSchneider/german_politic_direction_EuroBERT-210m")

In [ ]:
politic_pipeline = pipeline(
        "text-classification",
        model=model,
        tokenizer=tokenizer,
        top_k=None,
        device="cuda",
        trust_remote_code=True
    )

In [ ]:
party_vectors = {'DIE LINKE': [-1.0, 0.0],
 'BÜNDNIS 90/DIE GRÜNEN': [-0.9077674463309972, 0.4194737934385177],
 #'SPD': [-0.8074405688999996, 0.5899489195637577],
 'FDP': [0.0, 1.0],
 'CDU/CSU': [0.614758038308033, 0.788715762702673],
 'AfD': [1.0, 0.0]}

In [ ]:
def getPoliticalDirection(text):
    result = {list(x.values())[0]: list(x.values())[1] for x in politic_pipeline(text)[0]}
    result = np.sum([[result[key]*value[0], result[key]*value[1]] for key, value in zip(party_vectors.keys(), party_vectors.values())], axis=0)
    if result[1] == 0.0:
        result[1] = 0.00001
    result = math.atan(result[0]/result[1])/(np.pi/2)
    return result

In [ ]:
politic_pipeline("Heute Morgen war ich in Berlin-Marienfelde bei den Warnstreiks der Metaller. Wenn das Management den Strukturwandel verschläft, braucht es mindestens Lohnerhöhungen und Sicherheiten für die Beschäftigten.")

In [ ]:
text = "Auf unseren deutschen Autobahnen soll ein Tempolimit von 130 geben"
#text = "Deutschland den Deutschen, Ausländer raus"
#text = "Alle Schlüsselindustrien sollten verstaatlicht werden"
#text = "Wer langfristig in Wertpapieren spart - und wenn es nur 20 Euro im Monat sind - der soll am Ende den Veräußerungsgewinn nach einigen Jahren steuerfrei haben, damit sich für Millionen Wertpapiere lohnen und nicht nur für Millionäre."

In [ ]:
politic_pipeline(text)

In [ ]:
text = """
Lange war es still um Brasiliens Ex-Präsident Jair Bolsonaro – nun ist er zurück auf der Straße. Doch seine „große Rückkehr“ geriet am Sonntag zum großen Flop. Statt der von Bolsonaro selbst angekündigten „Million“ kamen laut Medienberichten gerade einmal 18.000 Menschen an den Strand von Copacabana. Einige sprechen von einer „politischen Beerdigung“.
Natürlich bleibt die radikale Rechte in Brasilien mobilisierungsfähig. Doch der jüngste Auftritt offenbarte mehr Schwächen als Stärke: Bolsonaro hat keine neuen Ideen, setzt auf altbekannte Diskurse: die Opferrolle, gepaart mit Angriffen gegen die Justiz. Doch wer Wahlen gewinnen will, muss mehr bieten. Vor allem muss er jene „schweigende Mehrheit“ erreichen, die ihn 2018 in die Regierung gebracht hatte. Davon war an diesem Wochenende kaum etwas zu spüren.
Auch die Strategie, sich an Trump zu orientieren, stößt an Grenzen. Während der US-Präsident trotz zahlreicher Skandale seinen Rückhalt in der republikanischen Partei behauptet, kämpft Bolsonaro zunehmend mit seiner politischen Bedeutungslosigkeit. Zwar gibt es bislang keinen Nachfolger mit ähnlicher Strahlkraft, doch Bolsonaro selbst wirkt zunehmend verbraucht. Bislang ist es ihm zwar gelungen, die Reihen seiner An­hän­ge­r*in­nen weitgehend geschlossen zu halten. Doch es ist gut möglich, dass der Konkurrenzkampf um die Führung der Rechten in Brasilien erst richtig beginnt, sobald Bolsonaro endgültig aus dem Rennen ist. Dafür könnte schon bald die Justiz sorgen.
Sollte sich der Verdacht bestätigen, dass Bolsonaro und seine Vertrauten tatsächlich einen Staatsstreich inklusive Mordfantasien gegen Präsident Lula planen, dürfte sein politischer Absturz kaum noch aufzuhalten sein. Ein wegen Putschversuchs und Anstiftung zum Mord angeklagter Bolsonaro wäre selbst für viele seiner bisherigen Wäh­le­r*in­nen nicht mehr tragbar. Wer in Brasilien langfristig politisch erfolgreich sein will, muss auch die politische Mitte für sich gewinnen.
Tot ist Bolsonaro allerdings noch lange nicht – dafür sitzt das bolsonaristische Projekt zu tief in den Köpfen vieler Brasilianer*innen. Doch ein politischer Durchmarsch à la Trump ist derzeit nicht in Sicht.
"""

In [ ]:
politic_pipeline(text)

In [ ]:
text = """
Bis zum 7. Oktober 2023 war es in der linken Klimaschutzorganisation Ende Gelände weitgehend Konsens, als Organisation keine Position zum Israel-Palästina-Konflikt zu beziehen. Dieser Minimalkonsens wurde von einigen Beteiligten in einer E-Mail im Oktober 2023 als »Schweigen zum anhaltenden Genozid an der palästinensischen Bevölkerung« und »Mittäterschaft« verurteilt, was Rassismus schüre. Mit einer antirassistischen Haltung sei das Schweigen nicht vereinbar, monierten die Kritiker.
Ein weiterer Vorwurf lautete, »der antideutsche Einfluss« würde »nicht nur die deutsche Linke weiter« spalten, sondern fördere »auch eine letztlich selbstzerstörerische Haltung«, die »die bereits aufstrebenden Kräfte der ex-tremen Rechten« stärke und es verhindere, »sinnvolle Solidarität mit Genossen international aufzubauen«. In Reaktion auf diese Kritik beschloss man bei Ende Gelände, sich mit dem Konflikt in Israel und Palästina auseinanderzusetzen, was eine Kleingruppe organisieren und anleiten sollte.
Die Resultate dieses Prozesses wurden im Dezember 2024 in einem Positionspapier mit dem Titel »Ende Gelände dem Schweigen« festgehalten. Darin kommt die Organisation nahezu allen Forderungen der Kritiker am vorherigen Konsens nach und geht sogar noch darüber hinaus.
In ihrem Papier macht sich Ende Gelände den gegen Israel gerichteten Vorwurf der Apartheid und des Genozids an Palästinensern zu eigen. Die Organisation erklärt sich solidarisch mit den »Palästinaprotesten« weltweit, legitimiert die antiisraelische Boykottkampagne BDS und lehnt die IHRA-Definition des Antisemitismus ab. Den Vorwurf des Antisemitismus erhebt die Organisation nun vielmehr gegen die Kritiker des Antisemitismus.
"""

In [ ]:
text = """
BERLIN. CDU-Chef Friedrich Merz hat die CDU-Schatzmeisterin Julia Klöckner als Bundestagspräsidentin vorgeschlagen. Die 52jährige würde damit die bisherige Präsidentin des Hohen Hauses, Bärbel Bas (SPD), ablösen, wie die Bild-Zeitung am Montag unter Berufung auf eine Sitzung des Fraktionsvorstands der Union berichtete. Der Bundestag muß der Personalie noch zustimmen, das gilt jedoch als sehr wahrscheinlich.
Klöckner hatte bei den jüngsten Bundestagswahlen ihren Wahlkreis im rheinland-pfälzischen Bad Kreuznach klar gegen den SPD-Kandidaten Joe Weingarten gewonnen. Im Bundestag gilt es als Tradition, daß die größte Fraktion auch den Bundestagspräsidenten stellen darf. Formell ist es das zweithöchste Amt im Staat – noch vor dem des Bundeskanzlers.
Klöckner gilt als migrationsskeptisch
Die Christdemokratin ist keine Unbekannte in der deutschen Politlandschaft. Sie trat 1996 in die CDU ein und wurde 2002 erstmals in den Bundestag gewählt. Zwischen 2010 und 2022 war sie Landesvorsitzende ihrer Partei in Rheinland-Pfalz, von 2011 bis 2018 außerdem Fraktionsvorsitzende der rheinland-pfälzischen CDU-Landtagsfraktion.
Im Februar 2016 äußerte sie Kritik an der Asylpolitik ihrer Parteifreundin und damaligen Bundeskanzlerin Angela Merkel. Zusammen mit dem CDU-Politiker Guido Wolf forderte sie tagesaktuelle Kontingente für Migranten sowie Grenzzentren. Außerdem sollte niemand ohne Asylgrund oder Schutzstatus nach Deutschland einreisen dürfen, unterstrichen Klöckner und Wolf damals. (st)
"""

In [ ]:
result = getPoliticalDirection(text)
pn.indicators.Gauge(name='', value=round(result, 4), bounds=(-1, 1), format=' ')

In [ ]:
result = getPoliticalDirection("")
pn.indicators.Gauge(name='', value=round(result, 4), bounds=(-1, 1), format=' ')